In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Generative AI Knowledge Base model predictions

Before you begin, make sure all the dependencies are installed.

In [ ]:
!pip install google-cloud-aiplatform google-cloud-firestore

## Overview

A **Large Language Model (LLM)** can be very good at answering general questions.
But it might not do as well to answer questions from your documents on its own.

The LLM will answer only from what it learned from its _training dataset_.
Your documents might include information or words that weren't on that dataset.
Or they might be used in a different or more specialized context.

This is where **Vector Search** comes into place.
Each time you upload a document, the Cloud Function webhook processes it.
When a document is processed, each individual page is _indexed_.
This allows us to not only find documents, but the specific pages.

The relevant pages can then be used as _context_ for the LLM to answer the question.
This _grounds_ the model to answer questions based on the documents only.
Without this, the model might give wrong answers, or _hallucinations_.

## My Google Cloud resources

Fill in your project ID, the
[Google Cloud location](https://cloud.google.com/about/locations)
you want to use, and your
[Vector Search index endpoint ID](https://console.cloud.google.com/vertex-ai/matching-engine/index-endpoints).

> 💡 The Vector Search index endpoint ID looks like a number, like `1234567890123456789`.

Run the following cell to set up your resources and authenticate to your account.

In [1]:
# @title
from google.colab import auth

project_id = "" # @param {type:"string"}
location = "us-central1" # @param {type:"string"}
index_endpoint_id = "" # @param {type:"string"}
deployed_index_id = "deployed_index" # @param {type:"string"}

auth.authenticate_user(project_id=project_id)

The first step is to initialize the Vertex AI client library using the location of your choice.

In [2]:
from google.cloud import aiplatform

aiplatform.init(location=location)

## Get text embeddings

You can use the Gecko model to get embeddings from text.
For more information, see the
[Get text embeddings](https://cloud.google.com/vertex-ai/generative-ai/docs/embeddings/get-text-embeddings)
page.

In [3]:
from vertexai.language_models import TextEmbeddingModel

def get_text_embedding(text: str) -> list[float]:
    model = TextEmbeddingModel.from_pretrained("textembedding-gecko@003")
    return model.get_embeddings([text])[0].values


# Convert the question into an embedding.
question = "What are LFs and why are they useful?"
question_embedding = get_text_embedding(question)
print(f"Embedding dimensions: {len(question_embedding)}")

Embedding dimensions: 768


## Find document context

All the documents you have processed have been indexed into your Vector Search index.
You can query for the closest embeddings to a given embedding from your Vector Search index endpoint.

> 💡 If you haven't processed any documents yet, you won't get any results.

In [4]:
from itertools import groupby

def find_document(question: str, index_endpoint_id: str, deployed_index_id: str) -> tuple[str, int]:
    # Get embeddings for the question.
    embedding = get_text_embedding(question)

    # Find the closest point from the Vector Search index endpoint.
    endpoint = aiplatform.MatchingEngineIndexEndpoint(index_endpoint_id)
    point = endpoint.find_neighbors(
        deployed_index_id=deployed_index_id,
        queries=[embedding],
        num_neighbors=1,
    )[0][0]

    # Get the document name and page number from the point ID.
    (filename, page_number) = point.id.split(':', 1)
    return (filename, int(page_number))

# Query the Vector Search index for the most relevant page.
(filename, page_number) = find_document(question, index_endpoint_id, deployed_index_id)
print(f"{filename=} {page_number=}")

filename='9410009v1.pdf' page_number=3


## Get document text

When documents were processed, their text was stored in Firestore as well.
The Vector Search query returned the relevant documents with their page numbers.
With this you can download the document's pages and give only the most relevant page to the model.

In [5]:
from google.cloud import firestore

def get_document_text(filename: str, page_number: int) -> str:
    db = firestore.Client(database='knowledge-base-database')
    doc = db.collection("documents").document(filename)
    return doc.get().get('pages')[page_number]

# Download the document's page text from Firestore.
context = get_document_text(filename, page_number)
print(f"{context[:1000]}\n...\n...")

EN SEM IND
FR SEM IND
VAR
REST {Magn( 1 )}
VAR
REST {Magn(
The interlingual status of the lexical function is
self-evident. Any occurrence of Magn will be left
intact during transfer and it will be the generation
component that ultimately assigns a monolingual
lexical entry to the LF.6
3.2 Problems
Lexical Functions abstract away from certain nu-
ances in meaning and from different syntactic re-
alizations. We discuss some of the problems raised
by this abstraction in this section.
Overgenerality An important problem stems
from the interpretation of LFs implied by their
use as an interlingua namely that the mean-
ing of the collocate in some ways reduces to the
meaning implied by the lexical function. This in-
terpretation is trouble-free if we assume that LFs
always deliver unique values; unfortunately cases
to the contrary can be readily observed. An exam-
ple attested from our corpus was the range of ad-
verbial constructions possible with the verbal head
oppose: adamantly, bitterly

## Ask a foundational model

With the relevant context ready, you can now make a _prompt_ that includes both the context and the question.

In [6]:
PROMPT = """\
CONTEXT:
{context}

QUESTION:
{question}
"""

This is the `text-bison`'s response.

In [7]:
from vertexai.language_models import TextGenerationModel

# Ask the foundational model.
model = TextGenerationModel.from_pretrained('text-bison')
response = model.predict(PROMPT.format(context=context, question=question))

print(f"{question}\n")
response.text.strip()

What are LFs and why are they useful?



'Lexical Functions (LFs) are abstract representations of the meaning of words and phrases. They are useful because they allow us to represent the meaning of words and phrases in a way that is independent of any particular language. This makes it possible to translate words and phrases between languages without having to worry about the specific grammatical rules of each language.\n\nFor example, the LF for the word "dog" might be something like "a four-legged mammal that barks". This LF can be used to translate the word "dog" into any other language, regardless of the specific grammatical rules of that language.\n\nLFs are also useful for representing'

## (Optional) Ask your tuned model

If you tuned a model, provide your tuned model ID.
You can find it in the [Vertex AI Model Registry](https://console.cloud.google.com/vertex-ai/models) by clicking on your tuned model and then navigating to the "Version details".

> 💡 The Model ID looks like a number, like `1234567890123456789`.

In [8]:
# @title
tuned_model_id = "" # @param {type:"string"}

This model was tuned with questions and answers generated with the [Gemini API](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/overview), so the responses will have a different tone.

In [9]:
# Ask the tuned model.
model = TextGenerationModel.get_tuned_model(tuned_model_id)
response = model.predict(PROMPT.format(context=context, question=question))

print(f"{question}\n")
response.text.strip()

What are LFs and why are they useful?



'LFs (Lexical Functions) are abstract representations of the meaning of words and phrases. They are useful because they allow for a consistent and interlingual approach to the translation of collocations and other lexical phenomena.'